# Numerical exercise - Quantum Mechanics
**By Martin Johnsrud**

## Theory  

In this exercise we study a wavefunction with a gaussian form,

$$
    \Psi(x, t) = \exp \bigg(- \frac{(x - x_0)}{2 \sigma_x^2} \bigg) \exp \big(i (k_0 x - \omega t) \big).
$$

obeying the relations 

$$
    \omega = \frac{\hbar k_0^2}{2m}, \quad E = \hbar \omega.
$$

## Implemetation

To make numerical approximations of quantum mechanical problems, we make it a discreet problem. We look at $N_x$ points at the $x$-axis, in the interval $[0, L]$, distributed equally with a disdance $\Delta x = L \,/ (N_x - 1)$, and time steps are made in descreet jumps as well. The wave function for a given time, $\Psi(x, t_0)$ is represented as a vector, $\pmb{\Psi^{(0)}} = \big[ \Psi_{j}\big]_{j = 0}^{N_x - 1}$, where the indecies corresponds to the value of the wave function at $j\Delta_x$, i.e. $\Psi_j = \Psi(j\Delta x)$. The Schrödinger equation can then be modeled using a discrete aproximation of the double derivative with a matrix $D^2$, such that

$$
    \big(D^2 \pmb{\Psi} \big)_j = \frac{\Psi_{j + 1} - 2  \Psi_{j} + \Psi_{j - 1}}{\Delta x ^2} 
$$

The Hilbert operator, 

$$
     \bar H =  - \frac{\hbar ^2}{2 m} \frac{\partial^2}{\partial x^2} + V(x),
$$

then becomes a matrix $\pmb{H}$ with elements

$$
    H_{j, j} = \frac{1}{m(\Delta x)^2} + V_j, \quad H_{j, j \pm 1} = -\frac{1}{2m(\Delta x)^2} \\
    H_{0, 0} = H_{N_x, N_x} = 0,
$$
Where the first and last elements make the boundary condittion. Time eveloution of the wave function from time $t_k$ to $t_{k + 1}$ then becomes
$$
    \pmb{\Psi^{(k + 1)}} = \pmb{\Psi^{(k)}} - i \Delta t \pmb{H} \pmb{\Psi^{(k)}}
$$

As a notational note, indecies $j$ refers to the discretisation of the spacial coordinate $x$, while indecies $k$ refers to the time coordinate. To increas the stability of the algorithm, the time evolution is split up into two equations, one for the real part of $\pm{\Psi}$, and one for the imaginary. The imaginary part, $\pmb{I^{(0)}}$, is initialized at time $t = 0$, while the real part $\pmb{R^{(0)}}$ is initialized at $t = \Delta t / 2$. The equations for time evolution then becomes

$$
    \pmb{I^{(k + 1)}} = \pmb{I^{(k)}} - i \Delta t \pmb{R^{(k)}} \\
    \pmb{R^{(k + 1)}} = \pmb{R^{(k)}} + i \Delta t \pmb{I^{(k + 1)}}
$$

## Notation
Correspondance between theory and code

**Parameters**  

$L, k_0, \hbar, m, N_x, \sigma_x, x_0, \Delta x, \Delta t, \omega$   
`L`, `k0`, `hbar`, `m`, `Nx`, `Sx`, `x0` `dx`, `dt`, `w`
 
`x` - Array of length $N_x$ representing the x-axis  
`H` - The matrix discretization of the hameltonian  
`V` - An array containing the values of the potential

In [ ]:
import numpy as np
from numpy import exp, sqrt 
from scipy import sparse
from matplotlib import pyplot as plt
from matplotlib import rcParams
from matplotlib.animation import FuncAnimation as FA
from mpl_toolkits.mplot3d import Axes3D
from IPython.display import HTML

font = {'family' : 'serif', 
        'weight' : 'normal', 
        'size'   : 18}
plt.rcParams['mathtext.fontset'] = 'dejavuserif'
plt.rc("lines", lw = 2)
plt.rc('font', **font)

L = 20
k0 = 20
hbar = 1
m = 1

Nx = 2000
x = np.linspace(0, L, Nx)
dx = x[1] - x[0]

dt = hbar / (hbar**2 / (2 * m) / dx**2) / 10
w = hbar * k0**2 / (2 * m)

print("dx = {:.4e}".format(dx))
print("dt = {:.4e}".format(dt))

In [ ]:
def getH(V):
    n = len(V)
    D1 =  hbar / m / dx**2 * np.ones(n) + V / hbar
    D1[0] = D1[-1] = 0
    D2 = -hbar / (2 * m) / dx**2 * np.ones(Nx)
    return sparse.dia_matrix(((D2, D1, D2), (-1, 0, 1)), shape = (n, n))

In [ ]:
def propagate(R, I, H, dt, steps):
    for i in range(steps):
        I = I - dt * H.dot(R)
        R = R + dt * H.dot(I)
    return (R, I)

def P(R, I):
    return R**2 + I**2

def expect(x, P):
    return x @ P

def initial(x, t, Sx, x0):
    return np.exp(-(x - x0)**2 / (2 * Sx**2)) * np.exp(1j * (k0 * x - w * t))

## Problem 1

In [ ]:
def getRI(dt, Sx = 1.5, x0 = 5):
    R = initial(x, dt / 2, Sx, x0).real
    I = initial(x, 0, Sx, x0).imag
    
    # Numerical normalization
    C = 1 / sqrt(np.sum(P(R, I)))
    
    return R * C, I * C

def plotWF(R, I):
    fig, ax = plt.subplots(figsize = (20, 5))
    ax.plot(x, R, label = "$R(x)$")
    ax.plot(x, I, label = "$I(x)$")
    ax.legend()
    plt.show()
    plt.close(fig)

def plotProb(p):
    fig, ax = plt.subplots(figsize = (20, 5))    
    ax.set_xlabel("$x$")
    ax.plot(x, p, label = "$|\Psi(x)|$")
    ax.legend()
    plt.show()
    plt.close(fig)

In [ ]:
V = np.zeros(Nx)
H = getH(V)
R, I = getRI(dt)
plotWF(R, I)
p = P(R, I)
plotProb(p)
print("Total probability is {}".format(sqrt(np.sum(p))))

## Problem 2
**Test of stability**

To test which $\Delta t$ gives a stable, we start with a maximum time step, `dt0`, and propagate a wavefunction $10$ steps with `dt` as different fractions of `dt0`, analtzing the outcome.

In [ ]:
dt0 = hbar / (hbar**2 / (2 * m) / dx**2 + np.max(V))
print(np.max(V))

n = 10
for i in range(1, 6):
    dt = dt0  * i / 20
    R, I = getRI(dt)
    print("Time step = dt0 * ", i / 20)
    R, I = propagate(R, I, H, dt, n)
    plotProb(P(R, I))

We see here that a timestep larger that 

$$
    \Delta t = \frac{\hbar}{\frac{\hbar^2}{2m}\frac{1}{\Delta x^2} + \max(V)} \frac{1}{10}
$$

leads to a jagged probbablitly function. In the rest of the caclulations, this value is used.

**Propagation**

The group velocity is given by

$$
    v_g = \frac{\partial \omega}{\partial k} \bigg\rvert_{k_0} = \frac{\hbar k_0}{m},
$$

so the time for the wave-packet to propagate a distance $l$, $T$, is given by

$$
    T = \frac{l}{v_g} = \frac{ml}{\hbar k_0}
$$

We propagate the wave from $x_0 = 5$ to $x = 15$, so that the time is given by $T = 10 m / \hbar k_0$, and the number of steps to take is

$$
    n = \frac{T}{\Delta t}
$$

In [ ]:
dt = hbar / (hbar**2 / (2 * m) / dx**2 + np.max(V)) / 10
T = 10 * m / (hbar * k0)
timeSteps = int(T / dt)

print("Time T = ", T)
print("Number of steps: ", timeSteps)

In [ ]:
def plotWF2(p0, p, Sx0):
    fig, ax = plt.subplots(1, figsize = (20, 5))
    ax.plot(x, p0, label = "$|\Psi(0, x)|^2$")
    ax.plot(x, p, label = "$|\Psi(t = T, x)|^2$")
    ax.set_xlabel("$x$")
    ax.set_ylabel("$|\Psi|^2$")
    ax.set_title("$\sigma_x = {}$".format(Sx))
    ax.legend()
    plt.show()
    plt.close(fig)

In [ ]:
Sxs = [0.5, 1.5]
for Sx in Sxs:
    R0, I0 = getRI(dt, Sx = Sx)
    R, I = propagate(R0, I0, H, dt, timeSteps)
    plotWF2(P(R0, I0), P(R, I), Sx)
    plotWF(R, I)

### Problem 3

The energy is given by
$$
    E = \hbar \omega
$$

In [ ]:
E0 = hbar * w
def getBarrierV(E0, l):
    if l == 0:
        return np.zeros(Nx), 0
    # Number of points in the barrier
    n = int(Nx / (L / l))
    # Number of points before and after the barrier
    n2 = int((Nx - n) / 2)
    # Makes sure the barrier has Nx points
    rest = Nx -  n - 2 * n2
    return np.concatenate((np.zeros(n2), E0 * np.ones(n), np.zeros(n2 + rest))), n2

In [ ]:
def plotWF3(p, V):
    fig, ax = plt.subplots(1, figsize = (20, 5))

    ax2 = ax.twinx()
    ax2.plot(x, V,  "r-", label = "$V(x)$")
    ax2.set_ylabel("$E / \hbar^2$")
    
    ax.plot(x, p, label = "$|\Psi(t = T, x)|^2$")
    ax.set_xlabel("$x$")
    ax.set_ylabel("$|\Psi|^2$")

    ax.legend(loc = "upper left")
    ax2.legend(loc = "upper right")
    plt.show()
    plt.close(fig)

In [ ]:
V, n2 = getBarrierV(E0 / 2, L / 20)
H = getH(V)
R, I = getRI(dt)
R, I = propagate(R, I, H, dt, timeSteps)
p = P(R, I)
plotWF3(p, V)
t = np.sum(p[-n2 - 1::])
print("Chance of transmission = {:.4e}".format(t))

### Problem 4


In [ ]:
Vs = np.linspace(0, 3 / 2 * E0, 50)
ts = np.zeros(len(Vs))
R0, I0 = getRI(dt)

for i in range(len(Vs)):
    V, n2 = getBarrierV(Vs[i], L / 20)
    H = getH(V)
    R, I = propagate(R0, I0, H, dt, timeSteps)
    p = P(R, I)
    ts[i] = np.sum(p[-n2 - 1::])
    
plt.plot(Vs, ts)
plt.show()

### Problem 5


In [ ]:
ls = np.linspace(0, L / 10, 50)
ts = np.zeros(len(ls))
R0, I0 = getRI(dt)

for i in range(len(ls)):
    V, n2 = getBarrierV(9 / 10 * E0, ls[i])
    H = getH(V)
    R, I = propagate(R0, I0, H, dt, timeSteps)
    p = P(R, I)
    ts[i] = np.sum(p[-n2 - 1::])
    
plt.plot(ls, ts)

## Miscellaneous

**Computational expence**

In [ ]:
from time import time
ts = []
ns = [a * 1000 for a in range(50)]
V = np.zeros(Nx)
dt = 1e-5
R0, I0 = getRI(dt)
H = getH(np.zeros(Nx))
for n in ns:
    t = time()
    propagate(R0, I0, H, dt, n)
    ts.append(time() - t)
    
plt.plot(ns, ts)
plt.show()

Wee can see that the time complexity of propagating the WF is only $O(n)$, even tough $\pmb{H}$ grows as $n^2$, thanks to `scipy` sparse matrix functionality.

**Animation**

In [ ]:
rcParams["animation.embed_limit"] =  "50"

# Psi[j, l, k] is Psi, evaluated at j dx, at time k dt, 
# where l = 0 gives real value, and l = 1 gives the imaginary value.

stepsPerFrame = 200
frames = 200
steps = stepsPerFrame * frames
R, I = getRI(dt)
H = getH(np.zeros(Nx))

Psi = np.zeros((timeSteps, 2, Nx))
Psi[0, 0] = R
Psi[0, 1] = I

print("Calculating...")
for i in range(1, frames):
    R, I = propagate(R, I, H, dt, stepsPerFrame)
    Psi[i, 0] = R
    Psi[i, 1] = I

fig = plt.figure(figsize=(15, 10))
ax = fig.add_subplot(111, projection = "3d")
l = [ax.plot(x, Psi[0, 0], Psi[0, 1])[0] for i in range(1)]

def anim(i, fargs):
    Psi = fargs
    l[0].set_data(x, Psi[i, 0])
    l[0].set_3d_properties(Psi[i, 1])
    return l

print("Aninmating...")
a = FA(fig, anim, fargs = (Psi,), frames = frames, interval = int(1000 / 30))

plt.close(fig)
HTML(a.to_jshtml())